# Text 생성 및 Prompt 예제

HyperCLOVAX‑SEED‑Text‑Instruct‑1.5B는 지시문 기반 텍스트-투-텍스트 모델로, 한국어 언어 및 문화 이해에 뛰어난 성능을 보입니다. 유사한 규모의 외부 경쟁 모델과 비교했을 때 수학적 성능이 향상되고, 한국어 능력이 크게 개선되었습니다. 이 모델은 HyperCLOVAX 시리즈에서 현재 출시된 모델 중 가장 작은 모델이며, 에지 디바이스와 같은 리소스가 제한된 환경에 적합한 경량 솔루션입니다. 최대 4K 토큰의 컨텍스트 길이를 지원하며, 다양한 작업에 적용 가능한 다목적 소형 모델입니다.

## 🔐 보안 설정 가이드

### 1. Hugging Face 토큰 생성
- https://huggingface.co/settings/tokens 에서 **Read 권한**의 토큰을 생성하세요.

### 2. 모델 접근 권한 요청
- 모델 페이지: https://huggingface.co/naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B
- **"Access repository"** 버튼을 클릭해서 권한 요청

### 3. 토큰 사용 방법

**방법 2: 런타임 입력**
- 노트북 실행 시 토큰을 직접 입력 (코드에 하드코딩하지 않음)


In [19]:
import os
import getpass
from huggingface_hub import login

def get_huggingface_token():
    """
    Hugging Face 토큰을 안전하게 가져오는 함수
    우선순위: 환경변수 > Colab secret > 사용자 입력
    """
    # 1. 환경변수에서 토큰 확인
    token = os.getenv('HUGGINGFACE_TOKEN')

    if token:
        print("✅ 환경변수에서 토큰을 찾았습니다.")
        return token

    # 2. Colab 환경에서 secret 사용 (Colab에서만 작동)
    try:
        from google.colab import userdata
        token = userdata.get('HUGGINGFACE_TOKEN')
        if token:
            print("✅ Colab secret에서 토큰을 찾았습니다.")
            return token
    except:
        pass  # Colab이 아닌 환경에서는 무시

    # 3. 사용자 직접 입력 (화면에 표시되지 않음)
    print("\n🔑 Hugging Face 토큰이 필요합니다.")
    print("토큰 생성: https://huggingface.co/settings/tokens")

    token = getpass.getpass("토큰을 입력하세요 (입력이 화면에 표시되지 않습니다): ").strip()

    if not token:
        raise ValueError("토큰이 입력되지 않았습니다.")

    if not token.startswith('hf_'):
        print("⚠️  토큰 형식을 확인해주세요. 'hf_'로 시작해야 합니다.")

    return token

def safe_login():
    """
    안전한 방식으로 Hugging Face에 로그인
    """
    try:
        token = get_huggingface_token()
        login(token=token)
        print("✅ Hugging Face 로그인 성공!")
        return True
    except Exception as e:
        print(f"❌ 로그인 실패: {e}")
        print("토큰을 확인하고 다시 시도해주세요.")
        return False

# 안전한 로그인 실행
login_success = safe_login()


🔑 Hugging Face 토큰이 필요합니다.
토큰 생성: https://huggingface.co/settings/tokens
토큰을 입력하세요 (입력이 화면에 표시되지 않습니다): ··········
✅ Hugging Face 로그인 성공!


In [20]:
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer
import torch

# 로그인 성공 여부 확인
if 'login_success' not in locals() or not login_success:
    print("❌ 로그인이 필요합니다. 먼저 위의 로그인 셀을 실행해주세요.")
    raise RuntimeError("Hugging Face 로그인이 필요합니다.")

print("🔄 모델을 로딩하는 중...")
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"

try:
    # GPU 사용 가능 여부 확인
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"📱 사용 중인 디바이스: {device}")

    # 모델과 토크나이저 로딩
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto" if device == "cuda" else None,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    print("✅ 모델 로딩 완료!")
    print(f"📊 모델 크기: {model.num_parameters():,} 파라미터")

except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")
    print("모델 접근 권한을 확인해주세요.")
    raise

🔄 모델을 로딩하는 중...
📱 사용 중인 디바이스: cuda
✅ 모델 로딩 완료!
📊 모델 크기: 1,585,547,264 파라미터


In [23]:
def clean_model_output(text):
    """
    모델 출력에서 불필요한 부분을 제거하는 함수
    """
    # tool_list, system, user 부분 제거
    lines = text.split('\n')
    cleaned_lines = []
    skip_until_assistant = False

    for line in lines:
        if line.strip() == "tool_list":
            skip_until_assistant = True
            continue
        elif line.strip() == "system":
            skip_until_assistant = True
            continue
        elif line.strip() == "user":
            skip_until_assistant = True
            continue
        elif line.strip() == "assistant":
            skip_until_assistant = False
            continue
        elif not skip_until_assistant:
            cleaned_lines.append(line)

    return '\n'.join(cleaned_lines).strip()

def generate_response(system_content, user_content, max_length=1024, repetition_penalty=1.2, clean_output=True, show_prompt=False):
    """
    HyperCLOVAX 모델을 사용하여 응답을 생성하는 함수

    Args:
        system_content (str): 시스템 메시지 내용
        user_content (str): 사용자 메시지 내용
        max_length (int): 최대 생성 길이 (기본값: 1024)
        repetition_penalty (float): 반복 패널티 (기본값: 1.2)
        clean_output (bool): 출력 정리 여부 (기본값: True)
        show_prompt (bool): 프롬프트 표시 여부 (기본값: False)

    Returns:
        str: 생성된 응답 텍스트
    """
    try:
        # 모델이 로드되었는지 확인
        if 'model' not in globals() or 'tokenizer' not in globals():
            return "❌ 모델이 로드되지 않았습니다. 먼저 모델 로딩 셀을 실행해주세요."

        # 프롬프트 표시 (디버깅용)
        if show_prompt:
            print("�� 시스템 프롬프트:", system_content)
            print("📝 사용자 입력:", user_content)
            print("-" * 50)

        chat = [
            {"role": "tool_list", "content": ""},
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
        ]

        inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_dict=True, return_tensors="pt")

        # 디바이스 설정
        device = "cuda" if torch.cuda.is_available() else "cpu"
        inputs = inputs.to(device)

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=max_length,
                repetition_penalty=repetition_penalty,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id
            )

        output_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

        # 필요시 <|endofturn|>, <|stop|> 등에서 자르기
        for stop_str in ["<|endofturn|>", "<|stop|>"]:
            if stop_str in output_text:
                output_text = output_text.split(stop_str)[0]

        # 출력 정리 (선택사항)
        if clean_output:
            output_text = clean_model_output(output_text)

        return output_text

    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"

## 문법 수정

SYSTEM : 당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다.  
USER :   
안갔어 시장에 그녀는.

In [24]:
# 문법 수정 예시
system_msg = "당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다."
user_msg = "안갔어 시장에 그녀는."

result = generate_response(system_msg, user_msg)
print(result)

시장에 안 갔어요. 그녀는요.


## 구조화되지 않은 데이터의 구문 분석
SYSTEM : 구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다.  
USER :   
최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다. 회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다. 네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데, 이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.

In [25]:
system_msg = "구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다."

user_msg = """
최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다.
회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다.
네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데,
이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.
이 데이터를 과일명, 색상, 맛으로 분석해 주세요.
"""

result = generate_response(system_msg, user_msg)
print(result)

다음은 최근 발견된 행성 구크럭스에서 발견된 다양한 과일에 대한 분석입니다.

| 과일명        | 색상       | 맛           |
|---------------|------------|--------------|
| 네오스키즐     | 보라색      | 사탕 맛       |
| 로험클         | 회색빛 파란색  | 시큼한 맛    |
| 포유닛        | 밝은 녹색   | 달콤하지만 풍미중시 |
| 루프노바          | 네온 핑크색   | 솜사탕같은 맛|
| 글로울            | 옅은 오렌지색 | 신맛 및 쓰림 |

이러한 정보는 각 과일의 특징을 이해하는 데 도움이 될 것입니다. 추가적인 연구와 탐사가 필요할 수도 있습니다.


## 대화 상태를 수동으로 관리

| 역할       | 설명                                                                                      |
|------------|-------------------------------------------------------------------------------------------|
| developer  | 애플리케이션 개발자가 제공한 지침. **user 메시지보다 우선적으로 처리**됩니다.              |
| user       | 최종 사용자가 입력한 지침. developer 메시지보다 **우선순위가 낮습니다.**                  |
| assistant  | 모델이 생성한 응답 메시지. 응답 결과는 이 역할로 반환됩니다.                              |


In [26]:
# 대화 히스토리를 하나의 시스템 메시지로 만들기
system_content = """이전 대화:
사용자: 안녕하세요.
어시스턴트: 안녕하세요. 무었을 도와드릴까요?
사용자: 딥러닝과 머신러닝의 차이점이 무었인가요?"""

user_content = "딥러닝과 머신러닝의 차이점에 대해 자세히 설명해주세요."

result = generate_response(system_content, user_content)
print(result)

안녕하세요! 딥러닝과 머신러닝은 모두 인공지능 분야에서 중요한 역할을 하지만, 그 목적이나 접근 방식에서 차이가 있습니다.

1. **목적**:
   - **머신러닝**: 주로 데이터로부터 패턴을 학습하여 예측 모델을 만드는 것을 목표로 합니다. 예를 들어, 스팸 이메일을 분류하거나 신용 카드가 사기인지 판단하는 등의 작업을 수행합니다.
   - **딥러닝**: 더 깊은 신경망을 사용하여 인간의 뇌와 유사한 방식으로 데이터를 처리하고 이해하는 것이 목표입니다. 이는 복잡한 문제를 해결하는 데 유용하며 이미지 인식, 자연어 처리 등 다양한 응용 분야에 적용됩니다.

2. **신경망 깊이와 구조**:
   - **머신러닝 모델(예: SVM, 로지스틱 회귀)**: 일반적으로 얕은 신경망을 사용합니다. 여러 레이어로 구성되어 있지만 각 층의 수는 제한적입니다.
   - **딥러닝 모델(예: CNN, RNN, LSTM)**: 깊은 신경망을 사용하므로 다수의 계층으로 구성되며, 이러한 구조를 통해 더욱 복잡한 문제도 해결할 수 있는 능력을 가집니다.

3. **학습 방법**:
   - **머신러닝 알고리즘**: 보통 정형화된 데이터에 잘 맞으며, 도메인 지식이 필요한 경우가 많습니다 (예: k-최근접 이웃(KNN), 의사결정 트리).
   - **딥러닝**: 비정규화 및 고차원 데이터까지 효과적으로 다룰 수 있으며, 훈련 과정이 상대적으로 복잡하지만 결과는 매우 강력할 수 있습니다. 특히 대규모 데이터셋에서도 일반화 능력이 뛰어납니다.

4. **응용 분야**:
   - **머신러닝**: 통계 분석, 금융 분석, 마케팅 자동화 등에서 많이 활용됩니다.
   - **딥러닝**: 자율 주행차, 음성 인식을 포함한 시각·청각 처리가 필요하거나 복잡한 문제를 다루는 경우 자주 선택됩니다.

결론적으로, 두 기술은 서로 보완적이며 특정 문제에 따라 적절한 방법을 선택할 필요가 있습니다. 어떤 경우에는 간단한 모델링으로도 충분하지만, 다른 문제는 심층적인 연구와 고급 기술을 요구하

## Keyword 추출

In [27]:
text = """
"블랙 온 블랙 도자기(Black-on-Black ware)는 뉴멕시코 북부의 푸에블로 원주민 도자기 예술가들이 개발한 20세기 및 21세기 도자기 전통입니다.
전통적인 환원 소성 블랙웨어는 푸에블로 예술가들에 의해 수세기 동안 만들어졌습니다.
지난 세기의 흑색 자기는 표면이 매끄럽고 선택적 버니싱이나 내화 슬립을 적용하여 디자인을 적용한 제품입니다.
또 다른 스타일은 디자인을 조각하거나 절개하고 융기된 부분을 선택적으로 연마하는 것입니다.
여러 세대에 걸쳐 Kha'po Owingeh와 P'ohwhóge Owingeh 푸에블로의 여러 가족은 여주인 도예가들로부터 전수받은 기술을 사용하여 검은 바탕에 검은 도자기를 만들어 왔습니다.
다른 푸에블로 출신의 예술가들도 검정색 바탕에 검정색 도자기를 제작했습니다. 몇몇 현대 예술가들은 조상의 도자기를 기리는 작품을 만들었습니다."
"""

instructions="텍스트 블록이 제공되며, 당신의 임무는 텍스트 블록에서 키워드 목록을 추출하는 것입니다."

result = generate_response(instructions, text)
print(result)

다음과 같은 주요 단어가 있습니다:

1. 블랙 온 블랙 (Black-on-Black)
2. 도자기 
3. 뉴멕시코
4. 푸에블로 원주민
5. 20세기
6. 21세기
7. 환원 소성
8. 디자인
9. 전통적인 기술
10. 검은 바탕
11. 예술가들
12. 조상
13. 작품


## Python 버그 수정



In [28]:
pgm_code = """
import Random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+ a +" x "+  b+"? "
    answer = input(question)
    if answer = a*b
        print (Well done!)
    else:
        print("No.")

"""

instructions="Python 코드 조각이 제공되며, 귀하의 임무는 그 안의 버그를 찾아 수정하는 것입니다. 수정된 코드를 제시해 주세요."

result = generate_response(instructions, pgm_code )
print(result)

코드에 몇 가지 오류가 있습니다:

- `answer` 변수에는 할당 연산자 대신 대입 연산자를 사용해야 합니다 (`==` 가 아니라).
- 반복문에서 `input()` 함수로 입력을 받고 있습니다만, 이 경우 각 시도마다 결과를 저장하고 비교할 수 있도록 변경되어야 합니다.
- 마지막 조건문이 잘못되었습니다: 문자열 연결과 정수 곱셈은 서로 다른 데이터 타입으로 인해 불가능합니다.

다음은 이러한 오류를 수정한 수정된 버전입니다:

```python
import random

random_a = random.randint(1, 11) # 범위 조정 (1~12 -> 2~10 또는 원하는 범위로 설정)

for _ in range(10): 
    while True: 
        try: 
            num_b = int(input(f"What is {random_a} x {num_b}? "))
            break
        except ValueError:
            print("Please enter a number.")
    
    result = random_a * num_b
    
    for i in range(5):  
        user_answer = input("Your result was: ")
        
        if str(result).lower() == user_answer.lower():
            print("Well done!")
            break  
        elif str(result).lower()!= user_answer.lower():  
            print("That's not correct.")
            
# 추가적인 로직 필요 시 여기에 작성하세요.
```

수정 사항 요약:
- 모든 출력 메시지를 적절하게 수정했습니다(`print ("...")` → `print("...")`, 예시 출력: 'Yo

## Program Code 설명

In [29]:
codes = """
class Log:
    def __init__(self, path):
        dirname = os.path.dirname(path)
        os.makedirs(dirname, exist_ok=True)
        f = open(path, "a+")

        # Check that the file is newline-terminated
        size = os.path.getsize(path)
        if size > 0:
            f.seek(size - 1)
            end = f.read(1)
            if end != "\n":
                f.write("\n")
        self.f = f
        self.path = path

    def log(self, event):
        event["_event_id"] = str(uuid.uuid4())
        json.dump(event, self.f)
        self.f.write("\n")

    def state(self):
        state = {"complete": set(), "last": None}
        for line in open(self.path):
            event = json.loads(line)
            if event["type"] == "submit" and event["success"]:
                state["complete"].add(event["id"])
                state["last"] = event
        return state
"""

instructions="당신에게는 코드 조각이 제공될 것이며, 당신의 임무는 그것에 한국어 주석을 추가하는 것입니다."

result = generate_response(instructions, codes)
print(result)

```python
import os
from uuid import uuid4 as UUID


# 로그 클래스 정의: 로깅 기능을 제공하는 클래스입니다.
class Log:

    # 초기화 메서드: 지정된 경로에 디렉토리를 생성하고 파일을 엽니다.
    
    def __init__(self, path): 
        dirname = os.path.dirname(path)  
        try:   
          os.makedirs(dirname, exist_ok=True)   
        except OSError as e:   
            print("Error creating directory %s: %s" % (dirname,e))   
        
        with open(path, "a+") as f:
            
            # 파일 크기를 가져와 마지막 줄로 사용합니다.
            size = os.path.getsize(path)

            # 파일의 끝에 세미콜론을 추가하여 형식을 유지합니다.
            if size > 0:
                f.seek(size - 1)
                end = f.read(1)
                if end!= ".":   
                    f.write(".")  

            # 현재 파일 포인터를 다시 설정합니다.
            self.f = f
            self.path = path
            
    # 'log' 함수: 이벤트를 로그에 기록합니다.

    def log(self, event):

        # 각 이벤트 ID가 고유하도록 UUID를 할당한 후 문자열로 변환합니다.
        event["_event_id"] = str(uuid.uuid4())

        # JSON 

## 감성 분류기
- 한개의 text 감성 분석

In [30]:
instruction="당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다."
input="나는 새로운 배트맨 영화가 좋습니다!"

result = generate_response(instruction,  input)
print(result)

긍정적인 감정입니다. "배트맨 영화"에 대한 긍정적인 표현을 사용하고 있습니다.


- 여러개의 text 감성 분석

In [31]:
sentiments = [
    "나는 새로운 배트맨 영화가 좋습니다!",
    "오늘 날씨가 너무 안 좋네요.",
    "커피 마시러 가는 길이 설레네요!",
    "회의가 너무 길고 지루했습니다.",
    "점심으로 먹은 피자가 맛있었어요."
]

instruction = "당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다."

responses = []
for sent in sentiments:
    result = generate_response(instruction, sent)
    # 감정 분석 결과 저장
    responses.append({
        "text": sent,
        "sentiment": result
    })

# 결과 출력
for result in responses:
    print(f"Sentiments: {result['text']}\nSentiment: {result['sentiment']}\n")

Sentiments: 나는 새로운 배트맨 영화가 좋습니다!
Sentiment: 긍정적인 감정입니다. "새로"라는 단어의 의미에 열정과 기대감이 담겨 있습니다. 또한 "좋습니다!"라고 표현하여 강한 호평을 나타내고 있습니다.

Sentiments: 오늘 날씨가 너무 안 좋네요.
Sentiment: 감정: 부정적

이유: "날씨가 너무 안 좋다"라는 표현은 일반적으로 부정적인 느낌을 주며, 불편하거나 우울한 기분을 나타낼 수 있습니다. 긍정적인 감정이 아닌 이유는 이러한 상황이 기분 전환에 방해가 되거나 일상 활동에 영향을 줄 가능성이 높기 때문입니다.

Sentiments: 커피 마시러 가는 길이 설레네요!
Sentiment: 긍정적인 감정입니다.

이 문장은 커피를 사러 가는 것이 기대되고 즐거운 일임을 표현하고 있습니다. "설렌다"라는 단어는 긍정적인 감정을 나타내며, 새로운 경험이나 활동을 앞두고 있을 때 일반적으로 사용되는 단어입니다.

Sentiments: 회의가 너무 길고 지루했습니다.
Sentiment: 부정적입니다. 회의 자체와 시간이 길게 느껴졌기 때문입니다.

Sentiments: 점심으로 먹은 피자가 맛있었어요.
Sentiment: 긍정적인 감정입니다! 맛있는 음식을 즐긴 경험에 대해 기쁘고 만족스러운 느낌을 표현하셨습니다.



## 회의록 요약

SYSTEM : 회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.  

 -토론의 전반적인 요약  
 -행동항목(무엇을 해야 하는지, 누가 하는지)  
 -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.  

In [32]:
meeting_minutes = """
회의 날짜: 2050년 3월 5일
 미팅 시간: 오후 2시
 위치: 은하계 본부 회의실 3B

 참석자:
 - 캡틴 스타더스트
 - 퀘이사 박사
 - 레이디 네뷸라
 - 초신성 경
 - 혜성 씨

 오후 2시 5분에 캡틴 스타더스트가 회의를 소집했습니다.

 1. 새로운 팀원인 Ms. Comet에 대한 소개와 환영 인사

 2. Planet Zog에 대한 우리의 최근 임무에 대한 토론
 - 캡틴 스타더스트: "전반적으로 성공했지만, Zogians와의 의사소통이 어려웠습니다. 언어 능력을 향상시켜야 합니다."
 - 퀘이사 박사: "동의합니다. 즉시 Zogian-영어 사전 작업을 시작하겠습니다."
 - Lady Nebula: "Zogian 음식은 말 그대로 이 세상의 것이 아니었습니다! 우리는 배에서 Zogian 음식의 밤을 갖는 것을 고려해야 합니다."

 3. 7구역 우주 해적 문제 해결
 - 초신성 경: "이 해적들을 처리하려면 더 나은 전략이 필요합니다. 그들은 이번 달에 이미 세 척의 화물선을 약탈했습니다."
 - 스타더스트 선장: "그 지역의 순찰을 늘리는 것에 대해 스타빔 제독과 이야기하겠습니다.
 - 퀘이사 박사: "저는 우리 함선이 해적의 발각을 피하는 데 도움이 될 수 있는 새로운 은폐 기술을 연구하고 있습니다. 프로토타입을 완성하려면 몇 주가 더 필요할 것입니다."

 4. 연례 은하계 베이크오프 검토
 - Lady Nebula: "우리 팀이 대회에서 2위를 했다는 소식을 전해드리게 되어 기쁩니다! 우리 화성 머드 파이가 대박을 쳤어요!"
 - 혜성 씨: "내년에는 1위를 목표로 합시다. 제 생각에는 승자가 될 수 있을 것 같은 주피터 젤로의 비법이 있습니다."

 5. 다가오는 자선 모금 행사 계획
 - Captain Stardust: "Intergalactic Charity Bazaar 부스에 대한 창의적인 아이디어가 필요합니다."
 - Sir Supernova: "'Dunk the Alien' 게임은 어때요? 외계인 복장을 한 자원봉사자에게 사람들이 물 풍선을 던지게 할 수 있어요."
 - 퀘이사 박사: "승자에게 상금을 주는 '별 이름을 지어라' 퀴즈 게임을 준비할 수 있어요."
 - Lady Nebula: "좋은 아이디어입니다, 여러분. 이제 보급품을 모으고 게임을 준비합시다."

 6. 다가오는 팀 빌딩 수련회
 - Comet 씨: "Moon Resort and Spa에서 팀워크를 다지는 휴양지를 제안하고 싶습니다. 최근 임무를 마친 후 유대감을 형성하고 휴식을 취할 수 있는 좋은 기회입니다."
 - 캡틴 스타더스트: "환상적인 생각이군요. 예산을 확인해 보고 실현할 수 있는지 알아보겠습니다."

 7. 차기회의 안건
 - Zogian-English 사전 업데이트 (Dr. Quasar)
 - 클로킹 기술 진행 보고서(퀘이사 박사)
 - 7번 구역 순찰 강화 결과(캡틴 스타더스트)
 - 은하계 자선 바자회 최종 준비(전체)

 회의가 오후 3시 15분에 연기되었습니다. 다음 회의는 2050년 3월 19일 오후 2시에 은하계 본부 회의실 3B에서 열릴 예정입니다.
"""
instruction = """
          회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.
             -토론의 전반적인 요약
             -행동항목(무엇을 해야 하는지, 누가 하는지)
             -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.
      """
result = generate_response(instruction, meeting_minutes)
print(result)

**회의 요약**

1. **새로운 팀원 소개 및 환영**
   - 새 팀원인 Comet에게 환영 인사를 전함. 

2. **Planet Zog 관련 의제**
   - Zogians과의 소통 문제를 해결하기 위한 방안 논의 
   - 영어 학습 필요성에 따라 Zogian-영어 사전 작업 착수 결정.

3. **7구역 우주 해적 대응책 모색**
   - 과거 약탈 사건 회상하며 새로운 전략 강조 
   - 은폐 기술 개발 현황 설명 및 추가 노력 약속.

4. **연례 은하계 베이크오프 평가**
   - 화성의 수상작 공개 및 기대감 표현
   - 내년 목표 설정 및 성공적인 요리법 탐색 확인.

5. **자선 모금 행사 기획**
   - 다양한 이벤트 아이디어 제시 ('Dunk the Alien


## 이모티콘 번역
SYSTEM : 텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요.  
USER : 인공지능은 큰 가능성을 지닌 기술이다.

In [33]:
instruction = "텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요."
content = "인공지능은 큰 가능성을 지닌 기술이다."

result = generate_response(instruction, content)
print(result)

🤖✨💡🌐🔮📚🛠️


## 번역

In [34]:
instruction = "당신은 영어로 된 문장을 받게 될 것이고, 당신의 임무는 그것을 한국어와 동시에 일본어로 번역하는 것입니다."
content = "My name is Jane. What is yours?"

result = generate_response(instruction, content)
print(result)

안녕하세요! 저는 "Jane"입니다.

당신의 이름은 무엇인가요?

私の名前はジャーネです。あなたの名前は何ですか？


In [35]:
# 실습 과제
"""
위에서 학습한 프롬프트 엔지니어링 기법을 활용하여 다음 과제를 수행해보세요:

1. 자신만의 창의적인 프롬프트를 작성해보세요
2. 다양한 파라미터(max_length, repetition_penalty)를 조정해보세요
3. clean_output=False로 설정하여 원본 출력과 비교해보세요

예시:
- 창의적 글쓰기 도우미
- 코딩 문제 해결 도우미
- 학습 계획 수립 도우미
- 창의적 아이디어 브레인스토밍
"""


'\n위에서 학습한 프롬프트 엔지니어링 기법을 활용하여 다음 과제를 수행해보세요:\n\n1. 자신만의 창의적인 프롬프트를 작성해보세요\n2. 다양한 파라미터(max_length, repetition_penalty)를 조정해보세요\n3. clean_output=False로 설정하여 원본 출력과 비교해보세요\n\n예시:\n- 창의적 글쓰기 도우미\n- 코딩 문제 해결 도우미  \n- 학습 계획 수립 도우미\n- 창의적 아이디어 브레인스토밍\n'

## 실습: 위의 Prompt 내용을 수정해 가며 api 실행